# Part A: Leaf Node and In-place Operation

In [527]:
import torch

w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

try:
    w += 1.0  
except RuntimeError as e:
    print(e)

output.backward()

print(f"grad: {w.grad}")
print(f"w: {w}")

a leaf Variable that requires grad is being used in an in-place operation.
grad: tensor([3., 2.])
w: tensor([2., 3.], requires_grad=True)


In [528]:
w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

w = w + 1.0

output.backward()

print(w.grad)

None


C:\Users\kingv\AppData\Local\Temp\ipykernel_11428\575195260.py:9: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:494.)
  print(w.grad)


In [529]:
w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

output.backward()
print("Gradient:", w.grad)  

with torch.no_grad():
    w += 1.0          

print("Updated w:", w)

Gradient: tensor([3., 2.])
Updated w: tensor([3., 4.], requires_grad=True)


# Part B: (mini-project) Training

In [530]:
import numpy as np
import pandas as pd

In [531]:
data_df = pd.read_csv("../data/BTCUSDT.csv", sep='|', nrows=10, header=None, usecols=[0,4],
                       names=['timestamp', 'close'])
data_df['timestamp'] = pd.to_datetime(data_df['timestamp'], unit='s')
data_df.set_index('timestamp', inplace=True)
print(data_df)

                       close
timestamp                   
2017-08-17 04:00:00  4261.48
2017-08-17 04:01:00  4261.48
2017-08-17 04:02:00  4280.56
2017-08-17 04:03:00  4261.48
2017-08-17 04:04:00  4261.48
2017-08-17 04:05:00  4261.48
2017-08-17 04:06:00  4261.48
2017-08-17 04:07:00  4261.48
2017-08-17 04:08:00  4261.48
2017-08-17 04:09:00  4261.48


In [532]:
data_tensor = torch.tensor(data_df.values, dtype=torch.float32)
data_tensor, data_tensor.shape

(tensor([[4261.4800],
         [4261.4800],
         [4280.5601],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800]]),
 torch.Size([10, 1]))

In [533]:
w = torch.randn_like(data_tensor, requires_grad=True)

target = torch.tensor(100)

learning_rate = 1e-9

for i in range(20):
    y = w.T @ data_tensor

    loss = (y - target) ** 2

    loss.backward()

    print(f"Epoch: {i}, loss: {loss}")

    with torch.no_grad():
        w -= learning_rate * w.grad

    w.grad.zero_()

print("weights after training : ", w)

Epoch: 0, loss: tensor([[1.0877e+08]], grad_fn=<PowBackward0>)
Epoch: 1, loss: tensor([[44060300.]], grad_fn=<PowBackward0>)
Epoch: 2, loss: tensor([[17848558.]], grad_fn=<PowBackward0>)
Epoch: 3, loss: tensor([[7230337.5000]], grad_fn=<PowBackward0>)
Epoch: 4, loss: tensor([[2928966.5000]], grad_fn=<PowBackward0>)
Epoch: 5, loss: tensor([[1186504.8750]], grad_fn=<PowBackward0>)
Epoch: 6, loss: tensor([[480645.6562]], grad_fn=<PowBackward0>)
Epoch: 7, loss: tensor([[194706.7344]], grad_fn=<PowBackward0>)
Epoch: 8, loss: tensor([[78874.5859]], grad_fn=<PowBackward0>)
Epoch: 9, loss: tensor([[31951.5625]], grad_fn=<PowBackward0>)
Epoch: 10, loss: tensor([[12943.3955]], grad_fn=<PowBackward0>)
Epoch: 11, loss: tensor([[5243.1602]], grad_fn=<PowBackward0>)
Epoch: 12, loss: tensor([[2124.0037]], grad_fn=<PowBackward0>)
Epoch: 13, loss: tensor([[860.3967]], grad_fn=<PowBackward0>)
Epoch: 14, loss: tensor([[348.5295]], grad_fn=<PowBackward0>)
Epoch: 15, loss: tensor([[141.2012]], grad_fn=<Pow

# Part C: Advanced Indexing and Broadcasting

In [534]:
data_df2 = pd.read_csv("../data/BTCUSDT.csv", sep='|', nrows=1000, header=None, usecols=[1, 2, 3, 4, 5],
                       names=['open', 'high', 'low', 'close', 'volume'])
data_tensor2 = torch.tensor(data_df2.values, dtype=torch.float32)
print(data_tensor2[:5])

tensor([[4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 1.7752e+00],
        [4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 0.0000e+00],
        [4.2806e+03, 4.2806e+03, 4.2806e+03, 4.2806e+03, 2.6107e-01],
        [4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 1.2008e-02],
        [4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 1.4080e-01]])


In [535]:
high = data_tensor2[:, 1]
low = data_tensor2[:, 2]
close = data_tensor2[:, 3]

typical_price = (high + low + close) / 3
typical_price.shape

torch.Size([1000])

In [536]:
volume = data_tensor2[:, -1]
min_vol = torch.min(volume)
vol_normalized = (volume - min_vol) / (torch.max(volume) - min_vol)
vol_normalized.shape

torch.Size([1000])

In [537]:
bullish = data_tensor2[data_tensor2[:, 3] > data_tensor2[:, 0]]
bullish.shape

torch.Size([110, 5])

# Part D: Custom Modules

In [538]:
import torch.nn as nn


class LinearRewardNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1, 5))
        self.register_buffer('profit_target', torch.tensor([[100.0]]))

    def forward(self, x):
        y = x @ self.weights.T 
        return y
    

In [539]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [540]:
data_tensor2 = data_tensor2.to(device)
model = LinearRewardNet().to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8)

epochs = 20

for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()
    y_pred = model(data_tensor2)
    loss = criterion(y_pred, model.profit_target)
    print(f"epoch: {epoch}, loss: {loss}")
    loss.backward()
    optimizer.step()
    

epoch: 0, loss: 60522124.0
epoch: 1, loss: 16573596.0
epoch: 2, loss: 4538615.0
epoch: 3, loss: 1242926.625
epoch: 4, loss: 340426.96875
epoch: 5, loss: 93284.265625
epoch: 6, loss: 25606.3203125
epoch: 7, loss: 7073.2080078125
epoch: 8, loss: 1998.0361328125
epoch: 9, loss: 608.2025146484375
epoch: 10, loss: 227.62669372558594
epoch: 11, loss: 123.40994262695312
epoch: 12, loss: 94.86837768554688
epoch: 13, loss: 87.05130004882812
epoch: 14, loss: 84.91140747070312
epoch: 15, loss: 84.32617950439453
epoch: 16, loss: 84.16561126708984
epoch: 17, loss: 84.12134552001953
epoch: 18, loss: 84.1092300415039
epoch: 19, loss: 84.10565948486328


c:\Users\kingv\anaconda3\envs\pytorch_env\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [541]:
model.eval()
with torch.no_grad():
    predict = model(data_tensor2)

    print(predict[:5])

tensor([[98.0205],
        [95.2207],
        [96.0581],
        [95.2397],
        [95.4429]], device='cuda:0')


# Part E: Normalizing and Adam optimizer

In [542]:
data_norm = ((data_tensor2 - data_tensor2.mean(dim=0)) / data_tensor2.std(dim=0)).to(device)

In [547]:
class LinearRewardNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1, 5))
        self.bias = nn.Parameter(torch.zeros(1))
        self.register_buffer('profit_target', torch.tensor([[100.0]]))

    def forward(self, x):
        y = x @ self.weights.T + self.bias
        return y

In [550]:
model = LinearRewardNet().to(device)

critrion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=10)

epochs = 100
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()
    y_pred = model(data_norm)
    loss = critrion(y_pred, model.profit_target)
    print(f"epoch: {epoch}, loss: {loss}")
    loss.backward()
    optimizer.step()

epoch: 0, loss: 10008.4619140625
epoch: 1, loss: 9512.1865234375
epoch: 2, loss: 6499.46630859375
epoch: 3, loss: 5235.42626953125
epoch: 4, loss: 4450.0234375
epoch: 5, loss: 3137.924072265625
epoch: 6, loss: 1838.6861572265625
epoch: 7, loss: 1059.1693115234375
epoch: 8, loss: 791.76513671875
epoch: 9, loss: 644.3317260742188
epoch: 10, loss: 384.6661682128906
epoch: 11, loss: 117.71128845214844
epoch: 12, loss: 36.45444107055664
epoch: 13, loss: 193.6685333251953
epoch: 14, loss: 451.08465576171875
epoch: 15, loss: 628.8782958984375
epoch: 16, loss: 678.1686401367188
epoch: 17, loss: 678.7376098632812
epoch: 18, loss: 719.578369140625
epoch: 19, loss: 803.1793212890625
epoch: 20, loss: 852.3026733398438
epoch: 21, loss: 802.9795532226562
epoch: 22, loss: 672.394775390625
epoch: 23, loss: 532.4863891601562
epoch: 24, loss: 435.94793701171875
epoch: 25, loss: 374.0062255859375
epoch: 26, loss: 302.89031982421875
epoch: 27, loss: 203.14093017578125
epoch: 28, loss: 101.83050537109375
e